# Matrix Multiplication From the Foundations

In [ ]:
from pathlib import Path
import pickle, gzip, math, os, time, shutil, matplotlib as mpl, matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Download Data

In [ ]:
# Downloading the MNIST dataset with additional checks
MNIST_URL = 'https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')

path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

In [ ]:
from urllib.request import urlretrieve
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [ ]:
# Double checking the location of download
!ls -l data

In [ ]:
# Loading the data as a tuple of tuples
with gzip.open(path_gz, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [ ]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

We aren't allowed to use Numpy, Pandas, PyTorch this early on, we'll have to work with the standard Python toolkit.

In [ ]:
lst_1 = list(x_train[0])
vals = lst_1[200:222]
vals

In [ ]:
len(lst_1)

Since we can't work with matrices at the moment, we will need to convert our list of 784 elements into lists of 28x28. To do that, we can use `chunks`

In [ ]:
# Creating a function for chunks
def chunks(x, sz):
    # Loop through values from 0 to length of the list based on size
    # Yield(iterator) allows us to keep returning values till all elements in the
    # input have finished.
    for i in range(0, len(x), sz): yield x[i:i + sz]

In [ ]:
list(chunks(vals, 5))

In [ ]:
mpl.rcParams['image.cmap'] = 'gray'
plt.imshow(list(chunks(lst_1, 28)));

We can continue working with iterators using the library `itertools`

In [ ]:
from itertools import islice

In [ ]:
it = iter(vals)
islice(it, 5)

`islice` allows us to move through our data chunks based on the step value. Once there is no more data remaining in the chunks, it will return an empty list.

In [ ]:
list(islice(it, 5))

In [ ]:
list(islice(it, 5))

In [ ]:
list(islice(it, 5))

In [ ]:
list(islice(it, 5))

In [ ]:
list(islice(it, 5))

In [ ]:
list(islice(it, 5))

In [ ]:
it = iter(lst_1)
img = list(iter(lambda: list(islice(it, 28)), []))

In [ ]:
plt.imshow(img);

## Creating a Matrix and Tensor

In [ ]:
# Indexing into an image for demo purposes
img[20][15]

In [ ]:
# Let's create a class to work with matrices
# At first, this will only return the first and second indeces of an image
class Matrix:
    def __init__(self, xs): self.xs = xs
    def __getitem__(self, idxs): return self.xs[idxs[0]][idxs[1]]

In [ ]:
# Testing
m = Matrix(img)
m[20, 15]

We are now allowed to use the PyTorch `tensor` feature.

In [ ]:
from torch import tensor

In [ ]:
# Testing
tensor([1,2,3,4])

In [ ]:
# Loading image to Tensor
tens = tensor(img)
tens.shape, tens[20, 15]

In [ ]:
# Mapping train and valid to a tensor
x_train, y_train, x_valid, y_valid = map(tensor, (x_train, y_train, x_valid, y_valid))
x_train.shape

In [ ]:
imgs = x_train.reshape((-1, 28, 28))
imgs.shape

In [ ]:
plt.imshow(imgs[0]);

In [ ]:
# Indexing into our tensor
imgs[0, 20, 15]

In [ ]:
num_imgs, cols = x_train.shape
y_train, y_train.shape

In [ ]:
%time min(y_train), max(y_train)

In [ ]:
%time y_train.min(), y_train.max()

## On Random Numbers

Although we can use the random number generator in Python, we will opt to do it the hard way. 

This is based on the **Wichmann Hill** algorithm.

In [ ]:
# Creating our custom pseudo random number generator
rnd_state = None
def seed(a):
    global rnd_state
    a, x = divmod(a, 30268)
    a, y = divmod(a, 30306)
    a, z = divmod(a, 30322)
    rnd_state = int(x) + 1, int(y) + 1, int(z) + 1

In [ ]:
# Testing
seed(91737649164947)
rnd_state

In [ ]:
def rand():
    global rnd_state
    x, y, z = rnd_state
    x = (171 * x) % 30269
    y = (172 * y) % 30307
    z = (170 * z) % 30323
    rnd_state = x, y, z
    return (x / 30269 + y / 30307 + z / 30323) % 1.0

In [ ]:
rand(), rand(), rand()

In [ ]:
# Fork's a process, which returns 0 in the child and the child's process id
# in the parent. 
# Right now, we are seeing similar random numbers because both the parent and the child
# are copies of each other.
if os.fork(): 
    print(f'In parent: {rand()}')
else:
    print(f'In child: {rand()}')
    os._exit(os.EX_OK)

Let's see whether our go to libraries correctly re-initialize the random stream in the forked versions.

In [ ]:
# Checking PyTorch's version.
import torch
if os.fork(): 
    print(f'In parent: {torch.rand(1)}')
else:
    print(f'In child: {torch.rand(1)}')
    os._exit(os.EX_OK)

In [ ]:
# Checking Numpy
import numpy as np
if os.fork(): 
    print(f'In parent: {np.random.rand(1)}')
else:
    print(f'In child: {np.random.rand(1)}')
    os._exit(os.EX_OK)

In [ ]:
# Checking base Python
from random import random
if os.fork(): 
    print(f'In parent: {random()}')
else:
    print(f'In child: {random()}')
    os._exit(os.EX_OK)

Python's implementation is the only one that gets it right!!

In [ ]:
plt.plot([rand() for _ in range(100)]);

In [ ]:
plt.hist([rand() for _ in range(10000)]);

In [ ]:
%timeit -n 10 list(chunks([rand() for _ in range(7840)], 10))

In [ ]:
%timeit -n 10 torch.randn(784, 10)

## Matrix Multiplication

In [ ]:
# Change torch display settings
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)

In [ ]:
torch.manual_seed(1)
# Creating random numbers for the weights
weights = torch.randn(784, 10)
bias = torch.zeros(10)

`m1` will be a subset of the first 5 digits, on which we will carry out matrix multiplication operations.

In [ ]:
m1 = x_valid[:5] # This will give us 5 rows of 5 images which have been flattened out.
m2 = weights

m1.shape, m2.shape

In [ ]:
ar, ac = m1.shape
br, bc = m2.shape

(ar, ac), (br, bc)

Carrying out a matrix multiplication on `5x784` and `784x10` will give us a resulting tensor of `5x10` which is the outcome of multiplying and adding 784 pairs of digits.

In [ ]:
# Creating an empty 5x10 tensor.
t1 = torch.zeros(ar, bc)
t1

In [ ]:
t1.shape

Now, for the matrix multiplication itself, we will need to create nested loops which will add results of the matrix multiplication to the newly created tensor `t1`:
1. Loop through each row from `m1`
2. Loop through each column from `m2`
3. Loop through each pair from `m1` and `m2`

In [ ]:
for i in range(ar): # Each row in m1       ----> 5
    for j in range(bc): # Each column in m2----> 10
        for k in range(ac): # Each pair    ----> 784
            t1[i, j] += m1[i, k] * m2[k, j]

In [ ]:
t1

Lets put this into a function

In [ ]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape 
    # Creating an empty 5x10 tensor.
    c = torch.zeros(ar, bc)
    # Run multiplication loops
    for i in range(ar): # Each row in m1       ----> 5
        for j in range(bc): # Each column in m2----> 10
            for k in range(ac): # Each pair    ----> 784
                c[i, j] += a[i, k] * b[k, j]
    return c

In [ ]:
%time _ = matmul(m1, m2)

That is unacceptably slow for just...

In [ ]:
ar*bc*ac

..items, imagine running this kind of function on hundreds of thousands or millions of images.

This is why we need to leverage libraries which can allow us to program in Python - but can compile our operations at considerably faster speeds.

## Introducing `Numba` 

In [ ]:
from numba import njit

Based on the documentation, Numba is a JIT compiler for Python and is tailor made for code that utilizes NumPy arrays, functions and loops.

The use of decorators is the most common way to use Numba. **Once called, the Numba decorated code / function is compiled JIT at native machine code speed.**

For more details, visit the [documentation page](https://numba.pydata.org/numba-doc/latest/user/5minguide.html).

In [ ]:
@njit
def dot(a, b):
    res = 0.
    for i in range(len(a)): res += a[i] * b[i] #Inner most loop from the previous section.
    return res

In [ ]:
from numpy import array

In [ ]:
%time dot(array([1., 2, 3]), array([2., 3, 4]))

By introducing these changes, we've brought CPU and Wall times down from 824 ms and 823 ms to 257 ms and 350 ms respectively. However, this is still pretty slow. The reason for this is that for the first run, Numba needs to compile our code on top of the execution.

In [ ]:
%time dot(array([1., 2, 3]), array([2., 3, 4]))

Let's alter the `matmul()` function to factor in the newly created `dot()` function.

In [ ]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    # Create empty tensor
    c = torch.zeros(ar, bc)
    # Run updated loop
    for i in range(ar):
        for j in range(bc): c[i, j] = dot(a[i, :], b[:, j])
    return c           

In [ ]:
m1a, m2a = m1.numpy(), m2.numpy()

In [ ]:
# Let's use the FastAI library to test the performance improvements of this seemingly simple update.
from fastcore.test import *

In [ ]:
test_close(t1, matmul(m1a, m2a))

In [ ]:
test_close??

In [ ]:
%timeit -n 50 matmul(m1a, m2a)

By adding `@njit` to the third part of the matrix multiplication loop, we've effectively sped up the operation by around **2000x**

## Element-Wise Operations

This is a good place to introduce [APL (Array Programming Language)](https://tryapl.org/) which has

> ...a powerful, concise syntax, it lets you develop shorter programs that enable you to think more about the problem you're trying to solve than how to express it to a .

The code below, for element-wise operations was first tested in the APL interface and then applied in Python.

In [ ]:
a = tensor([10., 6, -4])
b = tensor([2., 8, 7])
a, b

In [ ]:
a + b

In [ ]:
a < b

An interesting fact that Jeremy shared was that there is no function for `mean` in APL. The language allows users to define such operations themselves and he used the following code to define the function:

> **mean <- +/÷≢**


In [ ]:
# Taking the mean of the binary results above, which we've already tested in APL
(a < b).float().mean()

What is _even more interesting_ is the fact that we can carry out pretty much any kind of mathematical operation in APL. Just take the example of creating a 3x3 matrix below:

> m ← 3 3 ⍴ ⍳9

which is pretty awesome, considering that the output is the same as the PyTorch version below.

In [ ]:
m = tensor([[1., 2, 3], [4., 5, 6], [7., 8, 9]])
m

Lets introduce the concept of Froebenius Norm, which is:

> The Frobenius norm, sometimes also called the Euclidean norm (a term unfortunately also used for the vector L^2-norm), is matrix norm of an `m×n` matrix A defined as the square root of the sum of the absolute squares of its elements...The Frobenius Norm can also be considered as a vector norm and is expressed as:

$$\| A \|_F = \left( \sum_{i,j=1}^n | a_{ij} |^2 \right)^{1/2}$$

In code, this boils down to...

In [ ]:
sf = (m * m).sum()
sf

In [ ]:
sf.sqrt()

And to implement this in APL, we can take the matrix `m` which we created earlier, followed by:
> sf <- +/,m*m
> sf*0.5

We need to multiply `sf` with 0.5 since APL doesn't have a square-root function.

With this additional knowledge, we can alter the `matmul()` even further by carrying out element-wise dot products.

In [ ]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc): c[i, j] = (a[i, :] * b[:, j]).sum()
    return c

In [ ]:
test_close(t1, matmul(m1, m2))

In [ ]:
%timeit -n 50 _ = matmul(m1, m2)

Which is slower than the Numba implementation, but orders of magnitude faster than the first version of `matmul()`. Lets see what using `torch.dot()` yeilds in terms of performance improvements.

In [ ]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc): c[i, j] = torch.dot(a[i, :], b[:, j])
    return c

In [ ]:
test_close(t1, matmul(m1, m2))

In [ ]:
%timeit -n 50 _ = matmul(m1, m2)

This is still slower than Numba!

## Broadcasting

### Broadcasting with a Scalar

In [ ]:
a

In [ ]:
a > 0

In [ ]:
# Which is the same as 
a > tensor([0., 0, 0])

In [ ]:
# Simple operations
a + 1

In [ ]:
m

In [ ]:
2 * m

### Broadcast a Vector to a Matrix

In [ ]:
# Creating a rank 1 tensor
c = tensor([10, 20, 30])
c

In [ ]:
m

In [ ]:
# Matrices can be added together
m + c

In [ ]:
# This also works both ways
c + m

These operations come from a slightly obscure concept called `expand_as()` which expands a tensor to the given size of another tensor.

In [ ]:
# Our 1x3 tensor expands to a 3x3 shape
t = c.expand_as(m)
t 

In [ ]:
torch.Tensor.expand_as?

In [ ]:
# We can check what resides in memory
t.storage()

In [ ]:
# Stride is the jump necessary to move to the next element
t.stride(), t.shape

We can index with the special value `None`. Alternatively, `unsqueeze()` also converts a 1-dimensional array into a 2-dimensional array (with one of the dimensions having the value 1).

Using `None` is more flexible than the `unsqueeze()` method.

In [ ]:
c.unsqueeze(0), c[None, :]

In [ ]:
c.shape, c.unsqueeze(0).shape

In [ ]:
# Output one row with a new unit axis
c.unsqueeze(1), c[:, None]

In [ ]:
# Triple dots will always insert a unit axis in a tensor, regardless of rank.
c[None].shape, c[..., None].shape

In [ ]:
# Another way to look at the expand_as() function.
c[:, None].expand_as(m)

In [ ]:
# Orientations of mathematical operations can be modified based on the above
m + c[:, None]

In [ ]:
# A simple flip changes the orientation of the sum.
m + c[None, :]

### Broadcasting Rules

Two dimensions are equal when:
1. They are equal or,
2. one of them is 1, which means the dimension is broadcasted to make it the same size.

Also, arrays do not need to have the same number of dimensions for them to be able to interact with each other.

In [ ]:
c[None, :], c[None, :].shape

In [ ]:
c[:, None], c[:, None].shape

In [ ]:
# This results in an outer product
c[None, :] * c[:, None]

In [ ]:
# Outer boolean operations
c[None, :] > c[:, None]

## Matmul with Broadcasting

We can now apply what we've learned about broadcasting to further speed up `matmul()`.

So, now we can grab a single digit which is a 784x1 matrix. Using `expand_as()` on digit will give us the same shape as out weight matrix. We can then multiply both matrices to get a 784x10 result.

In [ ]:
digit = m1[0]
digit.shape, m2.shape

In [ ]:
digit[:, None].shape

In [ ]:
digit[:, None].expand_as(m2).shape

In [ ]:
(digit[:, None] * m2).shape

Using this approach, we can add broadcasting to the `matmul()` function.

In [ ]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        # Take the ith row, all the cols and add an axis at the end
        # then multiply by b and sum it up
        c[i] = (a[i, :, None] * b).sum(dim=0)
    return c

In [ ]:
test_close(t1, matmul(m1, m2))

In [ ]:
%timeit -n 50 _ = matmul(m1, m2)

Let's test this out on the whole dataset, instead of just a mini-batch

In [ ]:
tr = matmul(x_train, weights)
tr

In [ ]:
tr.shape

In [ ]:
%time _ = matmul(x_train, weights)

## Einstein Summation

Compact representations of mathematical operations, like Einstein Summation, allow us to achieve considerable speed improvements over standard approaches. As this [blog](https://ajcr.net/Basic-guide-to-einsum/) states:

> The einsum function is one of NumPy’s jewels. It can often outperform familiar array functions in terms of speed and memory efficiency, thanks to its expressive power and smart loops. On the downside, it can take a little while understand the notation and sometimes a few attempts to apply it correctly to a tricky problem.

In [ ]:
# Going back to our m1 and m2 tensors
m1.shape, m2.shape

In the notation below:
- `i` is 5
- `k` is 784
- `j` is 10

The resulting vector is 5x784x10, which is:
- The original 5 rows of m1.
- The original 10 columns of m2.
- The 784 dimension is common to m1 and m2

In [ ]:
# using Einsum notation in torch, and btw this also applies to Numpy
mr = torch.einsum('ik,kj -> ikj', m1, m2)
mr.shape

In [ ]:
# Summing up
mr.sum(1)

An interesting point to note is that omitting a letter from the output means that values along that axis will be summed. So, that allows us to further simplify the code.

In [ ]:
# Case in point
torch.einsum('ik,kj -> ij', m1, m2)

In [ ]:
# Rewriting the matmul function
def matmul(a,b):
    return torch.einsum('ik,kj->ij', a, b)

In [ ]:
# Running test close to check if the einsum result is equal to the original
test_close(tr, matmul(x_train, weights), eps=1e-3)

In [ ]:
%timeit -n 5 _ = matmul(x_train, weights)

Which is considerably faster that even the broadcasting approach.

## PyTorch op

We can also use PyTorch's function / operator for matrix multiplication operations. The result, at least for this dataset is around the same as the Einsum implementation.

In [ ]:
test_close(tr, x_train@weights, eps=1e-3)

In [ ]:
%timeit -n 5 _ = torch.matmul(x_train, weights)

## CUDA - for warp speed!!

The beauty of parallel processing, using CUDA, is that we can define self contained functions which won't interact with any other operations.

We can build some intuition by rewriting the matmul() function to fill in only one number in a tensor. This effectively converts our matmul() into a kernel.

In [ ]:
# This version takes in an additional parameter called 'grid', which allows us to
# set the location (using coordinates) of the single output.
def matmul(grid, a, b, c):
    # Grid inputs, should be inside the bounds of the output tensor
    i, j = grid
    if i < c.shape[0] and j < c.shape[1]:
        # Start at 0
        tmp = 0.
        # Loop through all of the columns of a and the rows of b for i and j
        for k in range(a.shape[1]): tmp += a[i, k] * b[k, j]
        c[i, j] = tmp

In [ ]:
res = torch.zeros(ar, bc)
# Run a matmul calculation and the output should be added to coordinates (0,0)
matmul((0, 0), m1, m2, res)
res

Now, we need to launch the kernel and pass in the key parameters of the tensor grid.

In [ ]:
def launch_kernel(kernel, grid_x, grid_y, *args, **kwargs):
    # Loop through the rows of a
    for i in range(grid_x):
        # Loop through the columns of b
        for j in range(grid_y): kernel((i, j), *args, **kwargs) # Unpack as 3 separate args

In [ ]:
res = torch.zeros(ar, bc)
launch_kernel(matmul, ar, bc, m1, m2, res)
res

Now that we have some intuition around this topic let's actually do it in parallel.

In [ ]:
from numba import cuda

In [ ]:
# Rewriting the njit version of matmul() using cuda.
# This implementation has just one small variation compared to the original.
# The decorator compiles the following into GPU code.
@cuda.jit 
def matmul(a, b, c):
    # We call the grid here, specifying the number of dimensions 
    i, j = cuda.grid(2)
    if i < c.shape[0] and j < c.shape[1]:
        tmp = 0.
        for k in range(a.shape[1]): tmp += a[i, k] * b[k, j]
        c[i, j] = tmp

In [ ]:
# Lets make sure that the local gpu is working properly
# Peak wattage should be lower since we're using the standard ASUS profile.
!nvidia-smi

In [ ]:
r = np.zeros(tr.shape) # output tensor
m1g, m2g, rg = map(cuda.to_device, (x_train, weights, r)) # 3 items to be copied to gpu

In [ ]:
r.shape

In [ ]:
TPB = 16 # Threads per block.
rr, rc = r.shape
blocks_per_grid = (math.ceil(rr / TPB), math.ceil(rc / TPB)) # Operations assigned to blocks
blocks_per_grid

For CUDA JIT to work, we need to add [] brackets in the next cell, which contains the `blocks_per_grid` result from the last cell and the TPB as well.

In [ ]:
matmul[blocks_per_grid, (TPB, TPB)](m1g, m2g, rg) # This launches the kernel
r = rg.copy_to_host()
test_close(tr, r, eps=1e-3)

In [ ]:
%%timeit -n 10
matmul[blocks_per_grid, (TPB, TPB)](m1g, m2g, rg)
r = rg.copy_to_host()

Quite an improvement here. And different gaming profiles have a direct impact on the speed of calculations. For reference, I haven't enabled the "Ultimate" GPU mode.

We can go even faster by using PyTorch ops.

In [ ]:
# Copying tensors over to the GPU
m1c, m2c = x_train.cuda(), weights.cuda()

In [ ]:
r = (m1c @ m2c).cpu() # copying operations to the host

In [ ]:
%timeit -n 10 r = (m1c @ m2c).cpu()

The final version is close to 5 million times faster than the original version.